In [16]:
import requests
import pandas as pd
import datetime
import numpy as np
import time

# Construct the API URL with parameters
url = "https://gis.ccc.govt.nz/server/rest/services/OpenData/Furniture/FeatureServer/19/query?where=1%3D1&outFields=Type,SiteName,PhotographURL,SAPInternalReference&resultType=none&maxRecordCount=10000&outSR=4326&f=json"
params = {
    'where': '1=1',
    'outFields': 'Type,SiteName,PhotographURL,SAPInternalReference',
    'resultType': 'none',
    'maxRecordCount': '10000',
    'outSR': '4326',
    'f': 'json'
}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Extract the features from the response
    features = data.get('features', [])
    #print(features[10])
    
    # Extract attributes from each feature
    for feature in features:
        records = feature["attributes"]
        #date_int =  feature["attributes"]['LastEditDate']#datetime.datetime.fromtimestamp(feature["attributes"]['LastEditDate']/1000)
        #date_time = datetime.datetime.fromtimestamp(date_int/1000)
    
    records = [feature['attributes'] for feature in features]
    geometry = [feature['geometry'] for feature in features]
    
    # Create DataFrame
    df = pd.DataFrame(records)
    #del(df["LastEditDate"])
    #df["Last_Edit_Date"] = date
    for values in geometry:
        df['lon'] = values['x']
        df['lat'] = values['y']

    #df["Last_Edit_Date"] = pd.to_datetime(df["LastEditDate"], unit='ms').dt.date
    #del(df["LastEditDate"])
    df['SiteName'] = df['SiteName'].str.split(' - ').str.get(1)
    year = datetime.datetime.now().year
    month = datetime.datetime.now().month
    day = datetime.datetime.now().day
    df['DateSaved'] = datetime.date(year,month,day)
    # Print DataFrame
    print(df.head(4))
else:
    print("Error:", response.status_code)

    Type        SiteName                                      PhotographURL  \
0  Bench  Sparks Wetland  https://gismap.ccc.govt.nz/Image/ManMade/ParkA...   
1  Bench  Sparks Wetland  https://gismap.ccc.govt.nz/Image/ManMade/ParkA...   
2  Bench  Sparks Wetland  https://gismap.ccc.govt.nz/Image/ManMade/ParkA...   
3  Bench  Sparks Wetland  https://gismap.ccc.govt.nz/Image/ManMade/ParkA...   

   SAPInternalReference         lon        lat   DateSaved  
0  IE000000000011406964  172.570748 -43.489373  2024-05-21  
1  IE000000000011406963  172.570748 -43.489373  2024-05-21  
2  IE000000000011406965  172.570748 -43.489373  2024-05-21  
3  IE000000000011502320  172.570748 -43.489373  2024-05-21  


In [6]:
from flask import Flask, jsonify

app = Flask(__name__)


@app.route('/api/data', methods=['GET'])
async def get_data():
    # Return the entire DataFrame as JSON
    return "Response" #jsonify(df.to_dict(orient='records'))

# @app.route('/api/column/<column_name>', methods=['GET'])
# def get_column(column_name):
#     # Return the specified column as JSON
#     if column_name in df.columns:
#         return jsonify(df[column_name].tolist())
#     else:
#         return jsonify({'error': 'Column not found'}), 404

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Program Files\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
print(df["SAPInternalReference"].nunique())
print(len(df["Type"]))

2000
2000
